<a href="https://colab.research.google.com/github/kanovotn/sentiment-analysis-model-trainer/blob/master/notebooks/finetune_bert_for_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with DistilBert model

Find the full decription of this project in my blog: https://kanovotn.github.io/2024-05-01-sentiment-analysis-with-hugging-face/

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
# Login to hugging face account to be able to upload models to HF hub

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Function to compute accuracy when tuning the model

import evaluate
import numpy as np

def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [1]:
# Verify dropout parameter is present in the model
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
print(model.config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.40.2",
  "vocab_size": 30522
}



In [ ]:
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoConfig, DistilBertForSequenceClassification, DistilBertConfig
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

class ModelSentiment:
  """
    A class for data preparation and fine-tuning selected pre-trained model
  """

    def __init__(self, checkpoint):

        id2label = {0: "NEGATIVE", 1: "POSITIVE"}
        label2id = {"NEGATIVE": 0, "POSITIVE": 1}
        self.checkpoint = checkpoint
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        configuration = DistilBertConfig.from_pretrained(
            self.checkpoint,
            num_labels=2,
            id2label = {0: "NEGATIVE", 1: "POSITIVE"},
            label2id = {"NEGATIVE": 0, "POSITIVE": 1}
            )
        self.model = DistilBertForSequenceClassification.from_pretrained(self.checkpoint, config=configuration)

    def _tokenize_function(self, example):
        return self.tokenizer(example['text'], max_length=512, truncation=True)

    def tokenize_dataset(self, dataset):
        tokenized_dataset = dataset.map(self._tokenize_function, batched=True)

        return tokenized_dataset

    def load_and_split_dataset(self, dataset_name):
        # Split ratios
        train_split = 0.6
        test_split = 0.2

        # Load the dataset
        dataset_train = load_dataset(dataset_name, split="train")
        dataset_test = load_dataset(dataset_name, split="test")

        # Merge them and shuffle
        dataset_full = concatenate_datasets([dataset_train, dataset_test])
        print(dataset_full)
        # Shuffle the data with fixed seed to ensure the reproducibility of the dataset
        dataset_full = dataset_full.shuffle(seed=42).flatten_indices()

        # Calculate the number of samples for train, validate, and test
        total_samples = len(dataset_full)
        train_size = int(total_samples * train_split)
        test_size = int(total_samples * test_split)

        # Split the dataset
        dataset_train = dataset_full.select(range(train_size))
        dataset_test = dataset_full.select(range(train_size, train_size + test_size))
        dataset_validation = dataset_full.select(range(train_size + test_size, total_samples))

        return dataset_train, dataset_validation, dataset_test

    def fine_tune_model(self, tokenized_dataset_train, tokenized_dataset_validate):

        training_args = TrainingArguments(
            output_dir="distilbert-base-finetuned-sentiment",
            learning_rate=2e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=32,
            #gradient_accumulation_steps=4,
            num_train_epochs=3,
            weight_decay=0.01,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            push_to_hub=True,
            )

        #tokenized_dataset_train, tokenized_dataset_validate = self.load_and_preprocess_dataset("mteb/tweet_sentiment_extraction")
        trainer = Trainer(
            self.model,
            training_args,
            train_dataset=tokenized_dataset_train,
            eval_dataset=tokenized_dataset_validate,
            data_collator=self.data_collator,
            tokenizer=self.tokenizer,
            compute_metrics=compute_metrics,
        )

        trainer.train()

        return trainer

    def save_model(self, trainer):
        trainer.push_to_hub()

In [ ]:
# ModelSentiment class in  action
model = ModelSentiment("distilbert/distilbert-base-uncased")

dataset_train, dataset_validation, dataset_test = model.load_and_split_dataset("imdb")
tokenized_dataset_train = model.tokenize_dataset(dataset_train)
tokenized_dataset_validation = model.tokenize_dataset(dataset_validation)
tokenized_dataset_test = model.tokenize_dataset(dataset_test)

trainer = model.fine_tune_model(tokenized_dataset_train, tokenized_dataset_validation)


# Fine-tune model on glue sst2 dataset
#model = ModelSentiment("lyrisha/distilbert-base-finetuned-imdb-sentiment")
#dataset = load_dataset("glue", "sst2")
#tokenizer = AutoTokenizer.from_pretrained("lyrisha/distilbert-base-finetuned-imdb-sentiment")
#def tokenize_function(example):
#        return tokenizer(example['sentence'], max_length=512, truncation=True)
#tokenized_dataset = dataset.map(tokenize_function, batched=True)
#trainer = model.fine_tune_model(tokenized_dataset["train"], tokenized_dataset["validation"])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


In [ ]:
# Let's publish the trained mode on the HF hub
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/lyrisha/distilbert-base-finetuned-imdb-sentiment/commit/2fbb349d9d95908ebc58c3af3ff57cd0c218f24e', commit_message='End of training', commit_description='', oid='2fbb349d9d95908ebc58c3af3ff57cd0c218f24e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Inference on fine-tuned model in the held-out test dataset
import numpy as np
predictions = trainer.predict(tokenized_dataset_test)
preds = np.argmax(predictions.predictions, axis=-1)

metric = evaluate.load("accuracy")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9257}